In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
NORM = np.load('Heartbeats/Normal/N_beats.npy',allow_pickle = True)
ARRH = np.load('Heartbeats/Arrhythmia/A_beats.npy',allow_pickle = True)

print('loaded')
NORM = list(NORM)
ARRH = list(ARRH)
print('listed')

NORM = tf.ragged.constant(NORM)
ARRH = tf.ragged.constant(ARRH)
print('ragged')

loaded
listed
ragged


In [3]:
arrh_labels = np.array(tf.cast(np.array(np.ones(int(ARRH.shape[0]))),tf.float16))
norm_labels = np.array(tf.cast(np.array(np.zeros(int(NORM.shape[0]))),tf.float16))

In [4]:
all_data = tf.concat([NORM,ARRH],axis = 0)
all_labels = np.concatenate([norm_labels,arrh_labels],axis = 0)
# all_labels = list(all_labels)
# all_data = all_data.to_list()
print(type(all_data),type(all_labels))

<class 'tensorflow.python.ops.ragged.ragged_tensor.RaggedTensor'> <class 'numpy.ndarray'>


In [ ]:
data = tf.data.Dataset.from_tensor_slices((all_data,all_labels))

In [ ]:
def positional_encoding(length, depth):

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (500**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.sin(angle_rads)

    return tf.cast(pos_encoding, dtype=tf.float16)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.positional_encoding = positional_encoding(length=875, depth=d_model).numpy()

    def call(self, x):
        # length = tf.shape(x)[1]
        # temp = input_embeddings(one_hot(x,d_model),875,d_model)
        x = x + self.positional_encoding[tf.newaxis, :, :]
        return x

In [6]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [7]:
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [8]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
          tf.keras.layers.Dense(dff, activation='relu'),
          tf.keras.layers.Dense(d_model),
          tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [9]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()

        self.self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

In [10]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads,dff, vocab_size, dropout_rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(d_model=d_model)

        self.enc_layers = [
            EncoderLayer(d_model=d_model,
                         num_heads=num_heads,
                         dff=dff,
                         dropout_rate=dropout_rate)
            for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x):
        # `x` is token-IDs shape: (batch, seq_len)
        x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

        # Add dropout.
        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x)

        return x  # Shape `(batch_size, seq_len, d_model)`.

In [11]:
num_layers = 4
d_model = 512
dff = 2048
input_vocab_size = 512
num_heads = 8
dropout_rate = 0.1

In [12]:
class Equinox(tf.keras.Model):
    def __init__(self, *, num_layers, d_model, num_heads, dff, input_vocab_size, dropout_rate=0.1):#, target_vocab_size):
        super().__init__()
        self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)
    
        self.dense1 = tf.keras.layers.Dense(256)
        self.dense2 = tf.keras.layers.Dense(128)
        self.final_layer = tf.keras.layers.Dense(1)
    
    def call(self,inputs):
#         context, x  = inputs
#         context = self.encoder(context)
#         print(context,x)
        
        x = self.encoder(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        
        logits = self.final_layer(x)

In [17]:
Equinox_M = Equinox(num_layers = 6,d_model = 512,dff = 2048,input_vocab_size = 512,num_heads = 8,dropout_rate = 0.1)
# Equinox_M.build(input_shape = (875,512))
Equinox_M.compile(optimizer = tf.keras.optimizers.Adam(),metrics = ['accuracy'])
# Equinox_M.build(input_shape = (875,512))
Equinox_M(np.ones((875,512)))
Equinox_M.summary()

Model: "equinox_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_4 (Encoder)         multiple                  63019008  
                                                                 
 dense_180 (Dense)           multiple                  131328    
                                                                 
 dense_181 (Dense)           multiple                  32896     
                                                                 
 dense_182 (Dense)           multiple                  129       
                                                                 
Total params: 63,183,361
Trainable params: 63,183,361
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 50
batch_size = 32

In [ ]:
history = Equinox_M.fit(train_ds.batch(batch_size),epochs=epochs,validation_data=val_ds.batch(batch_size),shuffle=True)